In [14]:
# 라이브러리 불러오기
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# 예제 2.7의 get_financials() 함수를 사용 (Dapada apiKey 필요)

my_key = "dpd-fbfc2319-f01b8df52ed74fc25aa37ee5023d2377cb372b1fd48bc28db1f6563b05dbbbef"

# 특정 종목(stockCode)의 재무항목(indicatorName) 데이터를 가져오는 함수를 정의
def get_financials(stockCode, indicatorName, apiKey, consolidated=True, ttm=True):
    
    if consolidated:   # 연결 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByTTM?indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByCUR?indicatorName={indicatorName}&stockCode={stockCode}"       
            
    else:              # 별도 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByTTM?indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByCUR?indicatorName={indicatorName}&stockCode={stockCode}"       
           
    headers = {"Authorization": f"{apiKey}"}
    response = requests.get(url, headers=headers)
    result = response.json()
    return pd.DataFrame(result)

# 삼성전자(005930)의 연결 재무제표 (분기 기준)에서 '영업이익' 항목의 데이터 수집
df_samsung = get_financials(stockCode='005930', 
                            indicatorName='영업이익', 
                            apiKey=my_key, 
                            consolidated=True, 
                            ttm=False)

# 영업이익의 변화율을 계산
df_samsung['change'] = df_samsung['value'].pct_change()

# quarter 열을 인덱스로 설정
df_samsung = df_samsung.set_index('quarter')

# SK하이닉스(000660)의 연결 재무제표 (분기 기준)에서 '영업이익' 항목의 데이터 수집
df_skhynix = get_financials(stockCode='000660', 
                            indicatorName='영업이익', 
                            apiKey=my_key, 
                            consolidated=True, 
                            ttm=False)

# 영업이익의 변화율을 계산
df_skhynix['change'] = df_skhynix['value'].pct_change()

# quarter 열을 인덱스로 설정
df_skhynix = df_skhynix.set_index('quarter')

df_samsung

,value,change
quarter,,
2023-Q3,2433534000000,NaN
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [15]:
# 데이터 값 형식 지정(소수점 이하 둘째 자리, 천 단위, 소수점 표시)
df_samsung.style.format(precision=2, thousands=',', decimal='.')

,value,change
quarter,,
2023-Q3,"2,433,534,000,000",nan
2023-Q2,"668,547,000,000",-0.73
2023-Q1,"640,178,000,000",-0.04
2022-Q4,"4,306,131,000,000",5.73


In [16]:
# 데이터 값 형식 지정(열 이름 대문자로 변환)
df_samsung.style.format_index(str.upper, axis=1)

,VALUE,CHANGE
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [17]:
# 데이터 값 형식 지정( 행 인덱스를 소문자로 변환)
df_samsung.style.format_index(str.lower, axis=0)

,value,change
quarter,,
2023-q3,2433534000000,nan
2023-q2,668547000000,-0.725277
2023-q1,640178000000,-0.042434
2022-q4,4306131000000,5.726459


In [19]:
# 데이터 값 형식 지정(열 이름 재지정)
df_samsung.style.relabel_index(['영업이익', '영업이익증가율'], axis=1)

,영업이익,영업이익증가율
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [21]:
# 데이터 값 형식 지정(행 인덱스 재지정)
new_dates = ['2023.09', '2023.06', '2023.03', '2022.12']
df_samsung.style.relabel_index(new_dates, axis=0)

,value,change
quarter,,
2023.09,2433534000000,nan
2023.06,668547000000,-0.725277
2023.03,640178000000,-0.042434
2022.12,4306131000000,5.726459


In [23]:
# 연결하기(concat)
data_styler = df_samsung.style.format(precision=2, thousands=',', decimal='.')
stat_styler = df_samsung.describe().style.format(precision=2, thousands=',', decimal='.')

data_styler.concat(stat_styler)

,value,change
quarter,,
2023-Q3,"2,433,534,000,000",nan
2023-Q2,"668,547,000,000",-0.73
2023-Q1,"640,178,000,000",-0.04
2022-Q4,"4,306,131,000,000",5.73
count,4.00,3.00
mean,"2,012,097,500,000.00",1.65
std,"1,744,275,349,362.88",3.54
min,"640,178,000,000.00",-0.73
25%,"661,454,750,000.00",-0.38


In [24]:
# Styler 객체를 통해 원본 DataFrame 얻기
df_stat_original = stat_styler.data
df_stat_original

,value,change
count,4.000000e+00,3.000000
mean,2.012098e+12,1.652916
std,1.744275e+12,3.544275
min,6.401780e+11,-0.725277
25%,6.614548e+11,-0.383856
50%,1.551040e+12,-0.042434
75%,2.901683e+12,2.842013
max,4.306131e+12,5.726459


In [25]:
# 데이터 숨기기( 열 제외)
df_samsung.style.hide(['value'], axis=1)

,change
quarter,
2023-Q3,nan
2023-Q2,-0.725277
2023-Q1,-0.042434
2022-Q4,5.726459


In [30]:
# 데이터 숨기기(행 제외)
df_samsung.style.hide(['2023-Q1', '2022-Q4'], axis=0)

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277


In [31]:
# 조건부 포맷팅(각 열의 최댓값)
df_samsung.style.highlight_max(color='lightgreen')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [32]:
# 조건부 포맷팅(각 열의 최솟값)
df_samsung.style.highlight_min(color='orange')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [33]:
# 조건부 포맷팅(각 행의 최솟값)
df_samsung.style.highlight_min(color='lightblue', axis=1)

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [35]:
# 조건부 포맷팅(누락 데이터)
df_samsung.style.highlight_null(color='red')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [36]:
# 막대 그래프
df_samsung.style.bar(subset=['change'], color='#d65f5f')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459


In [37]:
# 삼성전자, SK하이닉스 영업이익 증가율 데이터 정리
df_opm = pd.concat([df_samsung['change'], df_skhynix['change']], axis=1)
df_opm.columns=['samsung', 'skhynix']
df_opm

,samsung,skhynix
quarter,,
2023-Q3,NaN,NaN
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032


In [38]:
# 모든 원소에 대해서 각각 스타일 적용
def show_negative(value, props=''):
    return props if value < 0 else None

style_opm = df_opm.style.map(show_negative, props='color:red;')
style_opm

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032


In [40]:
def highlight_min(style, props=''):
    return np.where(style == np.nanmin(style.values), props, '')

# 데이터프레임 전체 중에서 최솟값
style_opm_df = df_opm.style.apply(highlight_min,
                                  props='color:white;background-color:darkred',
                                  axis=None)
style_opm_df

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032


In [41]:
# 데이터프레임 각 열의 최솟값
style_opm_col = df_opm.style.apply(highlight_min,
                                   props='color:white;background-color:darkred',
                                   axis=0)
style_opm_col

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032


In [42]:
# 데이터프레임 각 행의 최솟값
style_opm_row = df_opm.style.apply(highlight_min,
                                   props='color:white;background-color:darkred',
                                   axis=1)
style_opm_row

C:\Users\c403\AppData\Local\Temp\ipykernel_9292\1021075756.py:2: RuntimeWarning: All-NaN slice encountered
  return np.where(style == np.nanmin(style.values), props, '')


,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032


In [45]:
# 조건에 따라 다르게 적용

def profit_condition(value):
    if value < 0:
        return 'Loss'
    elif value > 0:
        return 'Gain'
    else:
        return 'No'

def apply_condition(styler):
    styler.set_caption('Gain/Loss')
    styler = styler.format(profit_condition, subset=['change'])
    styler = styler.background_gradient(subset=['change'], axis=None, vmin=-10, vmax=10, cmap='RdYlBu_r')

    return styler

df_samsung.style.pipe(apply_condition)

,value,change
quarter,,
2023-Q3,2433534000000,No
2023-Q2,668547000000,Loss
2023-Q1,640178000000,Loss
2022-Q4,4306131000000,Gain


In [50]:
# HTML CSS 스타일링
cell_hover = {
    'selector': 'tr:hover',
    'props':[('background-color', "#ffffb3")]
}

stat_styler.set_table_styles([cell_hover])

,value,change
count,4.00,3.00
mean,"2,012,097,500,000.00",1.65
std,"1,744,275,349,362.88",3.54
min,"640,178,000,000.00",-0.73
25%,"661,454,750,000.00",-0.38
50%,"1,551,040,500,000.00",-0.04
75%,"2,901,683,250,000.00",2.84
max,"4,306,131,000,000.00",5.73
